In [28]:
# Importing libraries and packages
import geopandas as gpd
import pandas as pd
import numpy as np
import json

# Visualization
import matplotlib.pyplot as plt

# Modeling
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn import metrics

# Save models
import pickle

In [29]:
# Display Options
pd.set_option('display.max_columns', None)

## Random Forest Model trained on Fema Large without Outliers, without rentalAssistanceAmount

### Load the Training and Test Datasets - FL/TX and PR/NC

In [30]:
femaTrainData = pd.read_csv('../../../data/open-fema/repair-replace/FEMA-Large-Demographics-FL-TX-clean.csv')
print('There are {} records in our training dataset.'.format(len(femaTrainData)))

/Users/nitinserro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 3187285 records in our training dataset.


In [31]:
femaTrainData.columns

Index(['disasterNumber', 'damagedCity', 'damagedStateAbbreviation',
       'damagedZipCode', 'householdComposition', 'grossIncome', 'specialNeeds',
       'homeOwnersInsurance', 'floodInsurance', 'inspected', 'rpfvl',
       'habitabilityRepairsRequired', 'destroyed', 'waterLevel', 'floodDamage',
       'foundationDamage', 'foundationDamageAmount', 'roofDamage',
       'roofDamageAmount', 'tsaEligible', 'tsaCheckedIn',
       'rentalAssistanceEligible', 'rentalAssistanceAmount',
       'repairAssistanceEligible', 'repairAmount',
       'replacementAssistanceEligible', 'replacementAmount', 'sbaEligible',
       'renterDamageLevel', 'rentalAssistanceEndDate', 'rentalResourceCity',
       'rentalResourceStateAbbreviation', 'rentalResourceZipCode',
       'primaryResidence', 'personalPropertyEligible', 'ppfvl',
       'censusBlockId', 'censusYear', 'id', 'censusTractId', 'censusid',
       'tractid', 'tractname', 'county', 'state', 'below_poverty_rate',
       'median_earnings_total', 'une

In [35]:
femaTestData = pd.read_csv('../../../data/open-fema/repair-replace/FEMA-Large-Demographics-PR-clean.csv')
print('There are {} records in our PR test dataset.'.format(len(femaTestData)))

There are 1041745 records in our PR test dataset.


In [36]:
femaTestData.columns

Index(['disasterNumber', 'damagedCity', 'damagedStateAbbreviation',
       'damagedZipCode', 'householdComposition', 'grossIncome', 'specialNeeds',
       'homeOwnersInsurance', 'floodInsurance', 'inspected', 'rpfvl',
       'habitabilityRepairsRequired', 'destroyed', 'waterLevel', 'floodDamage',
       'foundationDamage', 'foundationDamageAmount', 'roofDamage',
       'roofDamageAmount', 'tsaEligible', 'tsaCheckedIn',
       'rentalAssistanceEligible', 'rentalAssistanceAmount',
       'repairAssistanceEligible', 'repairAmount',
       'replacementAssistanceEligible', 'replacementAmount', 'sbaEligible',
       'renterDamageLevel', 'rentalAssistanceEndDate', 'rentalResourceCity',
       'rentalResourceStateAbbreviation', 'rentalResourceZipCode',
       'primaryResidence', 'personalPropertyEligible', 'ppfvl',
       'censusBlockId', 'censusYear', 'id', 'censusTractId', 'censusid',
       'tractid', 'tractname', 'county', 'state', 'below_poverty_rate',
       'median_earnings_total', 'une

In [38]:
femaNCTestData = pd.read_csv('../../../data/open-fema/repair-replace/FEMA-Large-Demographics-NC-clean.csv')
print('There are {} records in our NC test dataset.'.format(len(femaNCTestData)))

There are 132336 records in our NC test dataset.


### Define a subset of columns

IHP: https://docs.google.com/document/d/1nu0yENGAWnoiMcTufxYnH7xwdh8NfFum9ni9IYiSIdk/edit#

Demographics: https://docs.google.com/document/d/1cpznnaIb5CE21I2RO8y2xRvZKS8StcP_JeXDW2mUIis/edit?ts=60319d34#heading=h.j8u0tgugtaw

In [39]:
ihp_exclude_cols = ['disasterNumber', 
                    'damagedCity', 
                    'damagedStateAbbreviation',
                    'damagedZipCode',
                    'grossIncome',
                    'foundationDamageAmount',
                    'roofDamageAmount',
                    'tsaCheckedIn',
                    'rentalAssistanceAmount',
                    'rentalAssistanceEligible',
                    'repairAmount',
                    'replacementAmount',
                    'haAmount',
                    'renterDamageLevel', 
                    'rentalAssistanceEndDate', 
                    'rentalResourceCity',
                    'rentalResourceStateAbbreviation', 
                    'rentalResourceZipCode',
                    'personalPropertyEligible', 
                    'ppfvl',
                    'censusBlockId', 
                    'censusYear', 
                    'id']
demo_exclude_cols = ['censusTractId', 
                     'censusid',
                     'tractid', 
                     'tractname', 
                     'county', 
                     'state',
                     'median_earnings_total',]
demo_dvi_col = ['dvi']
demo_rate_cols = ['below_poverty_rate',
                  'unemployed_labor_rate',
                  'built_1979_or_earlier_rate', 
                  'owner_occupied_rate']

### RandomForest

https://machinelearningmastery.com/random-forest-ensemble-in-python/

- The “max_samples” argument can be set to a float between 0 and 1 to control the percentage of the size of the training dataset to make the bootstrap sample used to train each decision tree.
- max_features argument and defaults to the square root of the number of input features. 
- The number of trees can be set via the “n_estimators” argument and defaults to 100.
- The maximum tree depth can be specified via the max_depth argument and is set to None (no maximum depth) by default.

In [40]:
def run_rf(df, frac, max_depth, max_samples, n_estimators, min_samples_leaf):
    # Sample the dataset
    df_train = df.sample(frac=frac) if frac < 1.0 else df    
        
    # Create test/train split
    X = df_train.loc[:, df_train.columns != 'repairReplacementAmount']
    y = df_train.loc[:, 'repairReplacementAmount']    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 42) 
  
    print('Shape of Training and Test inputs')
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    
    # Define the model
    model_rf = RandomForestRegressor(max_depth = max_depth, max_samples = max_samples, n_estimators = n_estimators, 
                                     min_samples_leaf = min_samples_leaf, random_state = 42)
    
    # Fit the model
    model_rf.fit(X_train, y_train)
    
    return (model_rf, model_rf.predict(X_test), y_test)

In [41]:
def run_rf_grid_search(df, frac, scoring):
    # Sample the dataset
    df_train = df.sample(frac=frac) if frac < 1.0 else df    
        
    # Create test/train split
    X = df_train.loc[:, df_train.columns != 'repairReplacementAmount']
    y = df_train.loc[:, 'repairReplacementAmount']    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 42)
    
    print('Shape of Training and Test inputs')
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    
    # RandomForestRegressor default model
    model_rf = RandomForestRegressor(random_state = 42)
    
    # Create the parameter grid
    param_grid_rf = {
        'bootstrap': [True],
        'max_samples': [0.8, 0.9, None],
        'max_depth': [8, 9, 10],
        'n_estimators': [75, 100, 125],
        'min_samples_leaf': [1, 5, 10]
    }
    
    # Instantiate the grid search model
    grid_search_rf = GridSearchCV(estimator = model_rf, param_grid = param_grid_rf, 
                                  scoring = scoring, cv = 3, n_jobs = -1, verbose = 2)
    
    # Fit the grid search to the data
    grid_search_rf.fit(X_train, y_train)
    
    print('Best params:\n', grid_search_rf.best_params_)
    
    # Predict using best model
    model_rf_best = grid_search_rf.best_estimator_
    
    return (model_rf_best, model_rf_best.predict(X_test), y_test)

In [42]:
def evaluate(y_test, y_pred):
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('R-squared:', metrics.r2_score(y_test, y_pred))
    print('\n')
    print('Total HA repair-replace amount actual:    ${:,.2f}'.format(y_test.sum()))
    print('Total HA repair-replace amount predicted: ${:,.2f}'.format(y_pred.sum()))
    print('\n')
    
    # Calculate the number of observations that are off by more than 20%
    results_df = pd.DataFrame({'actual': y_test, 'predicted': y_pred})
    results_df['diff'] = results_df['predicted'] - results_df['actual']    
    results_df['percent_diff'] = (abs(abs(results_df['predicted'] / (results_df['actual'])) - 1) * 100).where(results_df['actual'] > 0, 0)    
    print('Percentage of predictions that are off by more than 20%: {:.2f}'.format( 
          len(results_df[results_df['percent_diff'] > 20])/len(results_df) * 100))   

In [43]:
def predict(df, model):
    X_test = df.loc[:, df.columns != 'repairReplacementAmount']
    y_test = df.loc[:, 'repairReplacementAmount']    
  
    print('Shape of Training and Test inputs')    
    print(X_test.shape, y_test.shape)
    
    return (model.predict(X_test), y_test)  

### Create IHP-only Train and Test Datasets

In [44]:
femaTrainDf = femaTrainData[femaTrainData.columns[~femaTrainData.columns.isin(
    ihp_exclude_cols + demo_exclude_cols + demo_dvi_col + demo_rate_cols)]]

In [45]:
femaTrainDf.columns

Index(['householdComposition', 'specialNeeds', 'homeOwnersInsurance',
       'floodInsurance', 'inspected', 'rpfvl', 'habitabilityRepairsRequired',
       'destroyed', 'waterLevel', 'floodDamage', 'foundationDamage',
       'roofDamage', 'tsaEligible', 'repairAssistanceEligible',
       'replacementAssistanceEligible', 'sbaEligible', 'primaryResidence',
       'repairReplacementAmount', 'ownRent_Owner', 'ownRent_Renter',
       'ownRent_Unknown', 'residenceType_Apartment',
       'residenceType_Assisted Living Facility', 'residenceType_Boat',
       'residenceType_College Dorm', 'residenceType_Condo',
       'residenceType_Correctional Facility', 'residenceType_House/Duplex',
       'residenceType_Military Housing', 'residenceType_Mobile Home',
       'residenceType_Other', 'residenceType_Townhouse',
       'residenceType_Travel Trailer', 'residenceType_Unknown'],
      dtype='object')

In [46]:
femaTestDf = femaTestData[femaTestData.columns[~femaTestData.columns.isin(
    ihp_exclude_cols + demo_exclude_cols + demo_dvi_col + demo_rate_cols)]]

In [47]:
femaTestDf.columns

Index(['householdComposition', 'specialNeeds', 'homeOwnersInsurance',
       'floodInsurance', 'inspected', 'rpfvl', 'habitabilityRepairsRequired',
       'destroyed', 'waterLevel', 'floodDamage', 'foundationDamage',
       'roofDamage', 'tsaEligible', 'repairAssistanceEligible',
       'replacementAssistanceEligible', 'sbaEligible', 'primaryResidence',
       'repairReplacementAmount', 'ownRent_Owner', 'ownRent_Renter',
       'ownRent_Unknown', 'residenceType_Apartment',
       'residenceType_Assisted Living Facility', 'residenceType_Boat',
       'residenceType_College Dorm', 'residenceType_Condo',
       'residenceType_Correctional Facility', 'residenceType_House/Duplex',
       'residenceType_Military Housing', 'residenceType_Mobile Home',
       'residenceType_Other', 'residenceType_Townhouse',
       'residenceType_Travel Trailer', 'residenceType_Unknown'],
      dtype='object')

In [48]:
femaNCTestDf = femaNCTestData[femaNCTestData.columns[~femaNCTestData.columns.isin(
    ihp_exclude_cols + demo_exclude_cols + demo_dvi_col + demo_rate_cols)]]

### Grid Search
Parameter Grid:

    param_grid_rf = {
        'bootstrap': [True],
        'max_samples': [0.8, 0.9, None],
        'max_depth': [8, 9, 10],
        'n_estimators': [75, 100, 125],
        'min_samples_leaf': [1, 5, 10]
    }
    
Grid Search Params:

    scoring='neg_mean_squared_error'
    cv = 3    

In [17]:
model_rf_best, y_pred, y_test = run_rf_grid_search(femaTrainDf, frac = 1.0, scoring = 'neg_mean_squared_error')

Shape of Training and Test inputs
(2827910, 33) (2827910,)
(706978, 33) (706978,)
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best params:
 {'bootstrap': True, 'max_depth': 9, 'max_samples': 0.8, 'min_samples_leaf': 5, 'n_estimators': 125}


In [18]:
evaluate(y_test, y_pred)

Mean Absolute Error: 26.7803822903206
Mean Squared Error: 188147.74399838116
Root Mean Squared Error: 433.76000737548543
R-squared: 0.9622043125642697


Total HA Amount actual:    $221,268,210.44
Total HA Amount predicted: $221,320,822.97


Percentage of predictions that are off by more than 20%: 0.94


In [19]:
# Look at the distribution of predictions/actuals/errors
results_df = pd.DataFrame({'actual': y_test, 'predicted': y_pred})
results_df['errors'] = results_df['predicted'] - results_df['actual']
results_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
actual,706978.0,312.977505,2231.149597,0.000000,0.0,0.0,0.0,33300.000000
predicted,706978.0,313.051924,2185.125245,0.000000,0.0,0.0,0.0,31024.726161
errors,706978.0,0.074419,433.760308,-32106.262813,0.0,0.0,0.0,29747.467553


In [21]:
# Save the model
pickle.dump(model_rf_best, open('./models/random_forest.sav', 'wb'))

In [22]:
# Load saved model
model_rf_sav = pickle.load(open('./models/random_forest.sav', 'rb'))
model_rf_sav.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 9,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': 0.8,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 125,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

### RandomForest Model - Hyperparameters based on grid search
    max_depth = 9, max_samples = 0.8, n_estimators = 125, 
    min_samples_leaf = 5, random_state = 42

In [16]:
model_rf, y_pred, y_test = run_rf(femaTrainDf, frac = 1.0, max_depth = 9, max_samples = 0.8, 
                                  n_estimators = 125, min_samples_leaf = 5)

Shape of Training and Test inputs
(2549828, 33) (2549828,)
(637457, 33) (637457,)


In [17]:
evaluate(y_test, y_pred)

Mean Absolute Error: 29.014254828243384
Mean Squared Error: 205733.0373221685
Root Mean Squared Error: 453.5780388446607
R-squared: 0.961829224734453


Total HA repair-replace amount actual:    $217,284,763.35
Total HA repair-replace amount predicted: $216,980,986.66


Percentage of predictions that are off by more than 20%: 1.03


In [18]:
results_df = pd.DataFrame({'actual': y_test, 'predicted': y_pred})
results_df['errors'] = results_df['predicted'] - results_df['actual']
results_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
actual,637457.0,340.861836,2321.597189,0.000000,0.0,0.0,0.0,33300.000000
predicted,637457.0,340.385291,2271.401395,0.000000,0.0,0.0,0.0,30807.577915
errors,637457.0,-0.476545,453.578144,-32068.137934,0.0,0.0,0.0,30286.179613


In [19]:
model_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 9,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': 0.8,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 125,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

### Predict on Test (NC) using RandomForest Model

In [53]:
model_rf = pickle.load(open('./models/random_forest.sav', 'rb'))

In [54]:
y_pred, y_test = predict(femaNCTestDf, model_rf)

Shape of Training and Test inputs
(132336, 33) (132336,)


In [55]:
evaluate(y_test, y_pred)

Mean Absolute Error: 77.05291707209919
Mean Squared Error: 604172.3353926402
Root Mean Squared Error: 777.2852342561514
R-squared: 0.8831895851486424


Total HA repair-replace amount actual:    $55,016,024.02
Total HA repair-replace amount predicted: $62,062,632.26


Percentage of predictions that are off by more than 20%: 2.78


In [56]:
results_df = pd.DataFrame({'actual': y_test, 'predicted': y_pred})
results_df['errors'] = results_df['predicted'] - results_df['actual']
results_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
actual,132336.0,415.729839,2274.266087,0.000000,0.0,0.0,0.0,33235.660000
predicted,132336.0,468.977695,2353.923738,0.000000,0.0,0.0,0.0,30473.780843
errors,132336.0,53.247856,775.462149,-24057.542882,0.0,0.0,0.0,30156.967765


### Predict on Test (PR) using RandomForest Model

In [49]:
model_rf = pickle.load(open('./models/random_forest.sav', 'rb'))

In [50]:
y_pred, y_test = predict(femaTestDf, model_rf)

Shape of Training and Test inputs
(1041745, 33) (1041745,)


In [51]:
evaluate(y_test, y_pred)

Mean Absolute Error: 47.59104858895865
Mean Squared Error: 191504.36499848688
Root Mean Squared Error: 437.6121170608589
R-squared: 0.9678677168326351


Total HA repair-replace amount actual:    $529,331,656.32
Total HA repair-replace amount predicted: $530,199,037.63


Percentage of predictions that are off by more than 20%: 2.66


In [52]:
results_df = pd.DataFrame({'actual': y_test, 'predicted': y_pred})
results_df['errors'] = results_df['predicted'] - results_df['actual']
results_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
actual,1041745.0,508.120179,2441.286527,0.000000,0.0,0.0,0.0,33300.000000
predicted,1041745.0,508.952803,2386.039837,0.000000,0.0,0.0,0.0,30802.669320
errors,1041745.0,0.832623,437.611535,-31463.948669,0.0,0.0,0.0,28590.764088


### Generate Prediction Files using RandomForest Model predictions

In [34]:
# Append predictions to femaTestData
femaTestData['repairReplacementAmount_predicted'] = y_pred

In [35]:
femaTestData.columns

Index(['disasterNumber', 'damagedCity', 'damagedStateAbbreviation',
       'damagedZipCode', 'householdComposition', 'grossIncome', 'specialNeeds',
       'homeOwnersInsurance', 'floodInsurance', 'inspected', 'rpfvl',
       'habitabilityRepairsRequired', 'destroyed', 'waterLevel', 'floodDamage',
       'foundationDamage', 'foundationDamageAmount', 'roofDamage',
       'roofDamageAmount', 'tsaEligible', 'tsaCheckedIn',
       'rentalAssistanceEligible', 'rentalAssistanceAmount',
       'repairAssistanceEligible', 'repairAmount',
       'replacementAssistanceEligible', 'replacementAmount', 'sbaEligible',
       'renterDamageLevel', 'rentalAssistanceEndDate', 'rentalResourceCity',
       'rentalResourceStateAbbreviation', 'rentalResourceZipCode',
       'primaryResidence', 'personalPropertyEligible', 'ppfvl',
       'censusBlockId', 'censusYear', 'id', 'censusTractId', 'censusid',
       'tractid', 'tractname', 'county', 'state', 'below_poverty_rate',
       'median_earnings_total', 'une

In [37]:
# Drop demographics from FEMA file
femaTestData.drop(['below_poverty_rate', 'unemployed_labor_rate', 'median_earnings_total',
                                   'built_1979_or_earlier_rate', 'owner_occupied_rate'], axis=1, inplace=True)
                            
femaTestData.columns

Index(['disasterNumber', 'damagedCity', 'damagedStateAbbreviation',
       'damagedZipCode', 'householdComposition', 'grossIncome', 'specialNeeds',
       'homeOwnersInsurance', 'floodInsurance', 'inspected', 'rpfvl',
       'habitabilityRepairsRequired', 'destroyed', 'waterLevel', 'floodDamage',
       'foundationDamage', 'foundationDamageAmount', 'roofDamage',
       'roofDamageAmount', 'tsaEligible', 'tsaCheckedIn',
       'rentalAssistanceEligible', 'rentalAssistanceAmount',
       'repairAssistanceEligible', 'repairAmount',
       'replacementAssistanceEligible', 'replacementAmount', 'sbaEligible',
       'renterDamageLevel', 'rentalAssistanceEndDate', 'rentalResourceCity',
       'rentalResourceStateAbbreviation', 'rentalResourceZipCode',
       'primaryResidence', 'personalPropertyEligible', 'ppfvl',
       'censusBlockId', 'censusYear', 'id', 'censusTractId', 'censusid',
       'tractid', 'tractname', 'county', 'state', 'haAmount',
       'repairReplacementAmount', 'ownRent_Own

### Combine Demographics with PR Predictions (GisJoin)

In [38]:
# Load demographics data
demoDf = pd.read_csv("../../../data/census-tract/census-tract-demographics.csv")

In [39]:
# Select subset of variables
demoDf = demoDf[['tractid', 'gisjoin']]
demoDf.head(3)

,tractid,gisjoin
0,12001000200,G1200010000200
1,12001000301,G1200010000301
2,12001000302,G1200010000302


In [40]:
# Merge predictions with demographics - need GISJOIN for Tableau visualizations
femaTestData = pd.merge(femaTestData, demoDf, left_on='censusTractId', right_on='tractid', suffixes=('', '_y'))

# Remove common join column
femaTestData = femaTestData.drop(['tractid_y'], axis=1)

In [41]:
femaTestData.columns

Index(['disasterNumber', 'damagedCity', 'damagedStateAbbreviation',
       'damagedZipCode', 'householdComposition', 'grossIncome', 'specialNeeds',
       'homeOwnersInsurance', 'floodInsurance', 'inspected', 'rpfvl',
       'habitabilityRepairsRequired', 'destroyed', 'waterLevel', 'floodDamage',
       'foundationDamage', 'foundationDamageAmount', 'roofDamage',
       'roofDamageAmount', 'tsaEligible', 'tsaCheckedIn',
       'rentalAssistanceEligible', 'rentalAssistanceAmount',
       'repairAssistanceEligible', 'repairAmount',
       'replacementAssistanceEligible', 'replacementAmount', 'sbaEligible',
       'renterDamageLevel', 'rentalAssistanceEndDate', 'rentalResourceCity',
       'rentalResourceStateAbbreviation', 'rentalResourceZipCode',
       'primaryResidence', 'personalPropertyEligible', 'ppfvl',
       'censusBlockId', 'censusYear', 'id', 'censusTractId', 'censusid',
       'tractid', 'tractname', 'county', 'state', 'haAmount',
       'repairReplacementAmount', 'ownRent_Own

### Create Prediction Files

In [42]:
# Write predictions
femaTestData.to_csv("./predictions/FEMA-Large-PR-predictions.csv", index=False, encoding='utf-8')